In [ ]:
import pdfplumber
import pandas as pd
import re

In [ ]:

pdf_path = "mp-wallet_20250310202148_2ca2.pdf"

# Lista para armazenar as linhas extraídas
linhas_extraidas = []

# Abrir o PDF e extrair o texto linha por linha
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        texto = page.extract_text()
        if texto:
            linhas_extraidas.extend(texto.split("\n"))

# Regex para detectar início de uma transação (data XX-XX-XXXX)
regex_data = re.compile(r"^\d{2}-\d{2}-\d{4}")
regex_id_operacao = re.compile(r"\b\d{9,}\b")  # Detecta IDs de operação (9+ dígitos)
regex_valor = re.compile(r"R\$[ ]?[\-\d\.\,]+")  # Ajustado para capturar valores acima de 1000 corretamente

# Lista para armazenar transações reconstruídas corretamente
transacoes = []
linhas_tratadas = linhas_extraidas

for i, linha in enumerate(linhas_extraidas):
    linha = linha.strip()
    
    if regex_data.match(linha):
        if linha.count(" ") < 6:
            linha = linha.split(" ", 1)[0] + " " + linhas_tratadas[i-1] + " " + linhas_tratadas[i+1] + " " + linha.split(" ", 1)[1]
            transacoes.append(linha)
            linhas_tratadas[i] = linha
            linhas_tratadas[i-1], linhas_tratadas[i+1] = "a", "a"
        else:
            transacoes.append(linha)

# Regex ajustada para capturar corretamente transações formatadas
padrao = re.compile(r"(\d{2}-\d{2}-\d{4})\s+(.+?)\s+(\d{9,})\s+R\$[ ]?([\-\d\.\,]+)\s+R\$[ ]?([\-\d\.\,]+)")

dados_formatados = []
for transacao in transacoes:
    match = padrao.search(transacao)
    print("match: ", match)
    print("transacao: ", transacao)
    if match:
        data = match.group(1)
        descricao = match.group(2).strip()
        id_operacao = match.group(3)
        valor = match.group(4).replace(".", "").replace(",", ".").strip()
        saldo = match.group(5).replace(".", "").replace(",", ".").strip()
        dados_formatados.append([data, descricao, id_operacao, valor, saldo])

df_final = pd.DataFrame(dados_formatados, columns=["Data", "Descrição", "ID_Operação", "Valor", "Saldo"])

df_final["Valor"] = pd.to_numeric(df_final["Valor"], errors="coerce")
df_final["Saldo"] = pd.to_numeric(df_final["Saldo"], errors="coerce")

df_final.to_csv("extrato_bancario.csv", index=False, encoding="utf-8")
